In [1]:
import os
from transformers import DistilBertTokenizer, DistilBertModel
from pymilvus import connections, Collection, FieldSchema, CollectionSchema, DataType
import torch
from sentence_transformers import SentenceTransformer

# Initialize DistilBERT tokenizer and model
#tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')
#model = DistilBertModel.from_pretrained('distilbert-base-uncased')
#model = SentenceTransformer('sentence-transformers/sentence-t5-base')
#model = SentenceTransformer('all-MiniLM-L6-v2')
model = SentenceTransformer('multi-qa-MiniLM-L6-cos-v1')

def get_embedding(chunk):
    """Get the embedding vector for a chunk of text"""
    return model.encode(chunk)

c:\Python311\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
import json
from pymilvus import connections, FieldSchema, CollectionSchema, DataType, Collection, utility, db

from pymilvus import connections
connections.connect(
  alias="default",
  host='localhost',
  port='19530'
)

In [3]:
from pymilvus import Collection
collection = Collection("python_doc")      # Get an existing collection.
collection.load()


In [4]:
search_params = {
    "metric_type": "L2", 
}

In [11]:
#query_vector = get_embedding("how do i write a module in C or C++?")
query_vector = get_embedding("how do i use regular expressions in Python?")
#time the search
import time
start = time.time()
res = collection.search(
    data=[query_vector],
    anns_field="embedding",
    param=search_params,
    limit=10,
)
end = time.time()
#print the time in ms
print("Time to search: ", (end - start)*1000, "ms")

print(res)

Time to search:  3.999948501586914 ms
["['id: 3141, distance: 0.3555751442909241, entity: {}', 'id: 3160, distance: 0.3647070527076721, entity: {}', 'id: 3041, distance: 0.5579851865768433, entity: {}', 'id: 3475, distance: 0.6107275485992432, entity: {}', 'id: 3052, distance: 0.699018657207489, entity: {}', 'id: 31886, distance: 0.7414852380752563, entity: {}', 'id: 58368, distance: 0.7472877502441406, entity: {}', 'id: 3268, distance: 0.7857130765914917, entity: {}', 'id: 3043, distance: 0.8134196400642395, entity: {}', 'id: 31389, distance: 0.8148602247238159, entity: {}']"]


In [12]:
#append to a list the ids of the response considering the structure is["['id: 4917, distance: 0.6798818111419678, entity: {}', 'id: 10402, distance: 0.6981911659240723, entity: {}', 'id: 2335, distance: 0.707775354385376, entity: {}', 'id: 9312, distance: 0.7683041095733643, entity: {}', 'id: 9599, distance: 0.7878801822662354, entity: {}', 'id: 4622, distance: 0.8129417896270752, entity: {}', 'id: 2336, distance: 0.8555702567100525, entity: {}', 'id: 10408, distance: 0.8695672750473022, entity: {}', 'id: 11102, distance: 0.8719511032104492, entity: {}', 'id: 10665, distance: 0.8743786215782166, entity: {}']"]
#then we can use the ids to get the text from the files
ids = res[0].ids

print(ids)

[3141, 3160, 3041, 3475, 3052, 31886, 58368, 3268, 3043, 31389]


In [13]:
import json

def retrieve_sentences_from_ids(ids):
    # Load data from JSON file
    with open("sentence_data.json", 'r', encoding='utf-8') as json_file:
        data = json.load(json_file)

    # Filter sentences with the given IDs
    retrieved_sentences = [record['sentence'] for record in data if record['id'] in ids]

    # Print retrieved sentences
    for idx, sentence in enumerate(retrieved_sentences, 1):
        print(f"{idx}. {sentence}")


retrieve_sentences_from_ids(ids)


1. Regular Expression HOWTO
************************

Author:
   A.M. Kuchling <amk@amk.ca>


Abstract
^^^^^^^^

This document is an introductory tutorial to using regular expressions
in Python with the "re" module.
2. Introduction

Regular expressions (called REs, or regexes, or regex patterns) are
essentially a tiny, highly specialized programming language embedded
inside Python and made available through the "re" module.
3. In
these cases, you may be better off writing Python code to do the
processing; while Python code will be slower than an elaborate regular
expression, it will also probably be more understandable.
4. Using Regular Expressions

Now that we've looked at some simple regular expressions, how do we
actually use them in Python?
5. Regular expressions will often be written
in Python code using this raw string notation.
6. First, this is the worst collision between
   Python's string literals and regular expression sequences.
7. (The first edition covered Python's now-re

In [14]:
print("Number of elements in collection: ", collection.num_entities)
print("Is empty: ", collection.is_empty)

Number of elements in collection:  90167
Is empty:  False


In [ ]:
collection.release()